# Contextual Bandit Problem - 콘텍스트 밴딧 문제

가장 simple한 형태인 MAB 문제에서, State의 개념을 도입한 문제
1개의 밴딧이 아니라 여러 개의 밴딧에 대해 고려
각각의 밴딧에서 최선의 action을 선택하여 잠재적 보상을 최선으로 하는 것이 목표

MAB에서 구현한 네트워크를 확장하여 신경망을 이용, 상태를 입력으로 받아 액션을 출력하는 신경망
학습에는 Policy Gradient(정책 경사) Update 활용. 이를 통해 State와 Action의 매핑을 학습.
결국 네트워크에는 특정 상태의 맥락에서의 해당 action에 대한 값을 네트워크 가중치로 가지게 됨.
또한, 정책 Pi의 의미도 네트워크 가중치라는 맥락에서 출력 계층에서 선택된 Action을 의미하게 됨.



In [1]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np


C:\Users\night\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\night\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\night\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\night\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [6]:
class contextual_bandit():
    def __init__(self):
        self.state = 0
        # 밴딧들의 손잡이 목록을 작성. 각 밴딧은 각각 손잡이 4, 2, 1이 최적임
        self.bandits = np.array([[0.2,0,-0.1,-5],[0.1,-5,1.0,0.25],[-5,5,5,5]])
        self.num_bandits = self.bandits.shape[0]
        self.num_actions = self.bandits.shape[1]
        
    def getBandit(self):
        # 각각의 에피소드에 대해 랜덤한 상태를 반환
        self.state = np.random.randint(0,len(self.bandits))
        return self.state
    
    def pullArm(self,action):
        # 랜덤한 수를 얻는다.
        bandit = self.bandits[self.state,action]
        result = np.random.randn(1)
        if result > bandit:
            # 양의 보상을 반환한다.
            return 1
        else:
            # 음의 보상을 반환한다.
            return -1       

In [ ]:
class agent():
    def __init__(self, lr, s_size, a_size):
        # 네트워크의 피드포워드 부분, 에이전트는 상태를 받아서 액션을 출력한다.
        self.state_in= tf.placeholder(shape=[1],dtype=tf.int32)
        self_in_OH= slim.one_hot_encoding(self.state_in,s_size)
        output= slim.fully_conntected(state_in_OH, a_size,\
                                     biases_initializer=None, activation_fn=tf.nn.sigmoid,\
                                     weights_initializer=tf.ones_initializer())
        self.output = tf.reshape(output,[1])
        self.chosen_action = tf.argmax(self.output, 0)
        
        # 학습 과정을 구현한다.
        # 비용을 계산하기 위해 보상과 선택된 액션을 네트워크에 피드하고, 
        # 네트워크를 업데이트하는 데에 이를 이용한다.
        self.reward_holder = tf.placeholder(shape=[1],dtype=tf.float32)
        self.action_holder = tf.placeholder(shape=[1],dtype=tf.int32)
        self.responsible_weight = tf.slice(self.output, self.action_holder, [1])
        self.loss = -(tf.log(self.responsible_weight)*self.reward_holder)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=lr)
        self.update = optimizer.minimize(self.loss)
        